# Transfer Learning experiments

In [1]:
import os
import torch
import mlflow
import numpy as np
from torch import nn
from torch import optim
from collections import OrderedDict
import torch.nn.functional as F
from torchvision import datasets, transforms, models

## Transfer Learning with DenseNet

### Loading data 

In [2]:
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
                                      ])

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
                                    ])

# setting up data loaders
data_dir = os.path.join(os.pardir, 'data', 'Plant_leave_diseases_224')

train_data = datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=train_transforms)
test_data = datasets.ImageFolder(os.path.join(data_dir, 'validation'), transform=test_transforms)


### Getting Resnet model

In [3]:
model = models.densenet121(pretrained=True)

# Freezing the paramiters of the layers we do not want to train
for parameters in model.parameters():
    parameters.requires_grad = False

In [4]:
# Updating Classification layer 
_inputs = model.classifier.in_features

model.classifier = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(_inputs, 500)),
    ('relu', nn.ReLU()),
    ('dropout', nn.Dropout(0.2)),
    ('fc2', nn.Linear(500, 39)),
    ('output', nn.LogSoftmax(dim=1))
]))


### Training

In [5]:
# Configs 
config = {
    'max_epochs': 200,
    'learning_rate': 0.003,
    'resolution': 32
}

In [6]:
def train(model, train_loader, validation_loader, config, n_epochs=10, stopping_treshold=None):

    if torch.cuda.is_available():
        print('CUDA is available!  Training on GPU ...')
        model.cuda()


    # Loss and optimizer setup 
    criterion = nn.NLLLoss()
    optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])

    # Setting minimum validation loss to inf
    validation_loss_minimum = np.Inf 
    train_loss_history = []
    validation_loss_history = []

    for epoch in range(1, n_epochs +1):

        training_loss = 0.0
        validation_loss = 0.0

        # Training loop
        training_accuracies = []
        for X, y in train_loader:
            
            # Moving data to gpu if using 
            if torch.cuda.is_available():
                X, y = X.cuda(), y.cuda()
            
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(X)
            # calculate the batch loss
            loss = criterion(output, y)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            # update training loss
            training_loss += loss.item()*X.size(0)

            # calculating accuracy
            ps = torch.exp(output)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == y.view(*top_class.shape)
            training_accuracies.append(torch.mean(equals.type(torch.FloatTensor)).item())

        # Validation Loop
        with torch.no_grad():
            accuracies = []
            for X, y in validation_loader:

                # Moving data to gpu if using 
                if torch.cuda.is_available():
                    X, y = X.cuda(), y.cuda()
                # forward pass: compute predicted outputs by passing inputs to the model
                output = model(X)
                # calculate the batch loss
                loss = criterion(output, y)
                # update validation loss
                validation_loss += loss.item()*X.size(0)

                # calculating accuracy
                ps = torch.exp(output)
                top_p, top_class = ps.topk(1, dim=1)
                equals = top_class == y.view(*top_class.shape)
                accuracies.append(torch.mean(equals.type(torch.FloatTensor)).item())
                
        # Mean loss 
        mean_training_loss = training_loss/len(train_loader.sampler)
        mean_validation_loss = validation_loss/len(validation_loader.sampler)
        mean_train_accuracy = sum(training_accuracies)/len(training_accuracies)
        mean_accuracy = sum(accuracies)/len(accuracies)
        train_loss_history.append(mean_training_loss)
        validation_loss_history.append(mean_validation_loss)

        # Printing epoch stats
        print(f'Epoch: {epoch}/{n_epochs}, ' +\
              f'Training Loss: {mean_training_loss:.3f}, '+\
              f'Train accuracy {mean_train_accuracy:.3f} ' +\
              f'Validation Loss: {mean_validation_loss:.3f}, '+\
              f'Validation accuracy {mean_accuracy:.3f}')

        # logging with mlflow 
        if mlflow.active_run():
            mlflow.log_metric('loss', mean_training_loss, step=epoch)
            mlflow.log_metric('accuracy', mean_train_accuracy, step=epoch)
            mlflow.log_metric('validation_accuracy', mean_accuracy, step=epoch)
            mlflow.log_metric('validation_loss', mean_validation_loss, step=epoch)

        # Testing for early stopping
        if stopping_treshold:
            if mean_validation_loss < validation_loss_minimum:
                validation_loss_minimum = mean_validation_loss
            elif len([v for v in validation_loss_history[-stopping_treshold:] if v >= validation_loss_minimum]) >= stopping_treshold:
                print(f"Stopping early at epoch: {epoch}/{n_epochs}")
                break
        

    return train_loss_history, validation_loss_history

In [7]:

train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
validation_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=True)

mlflow.set_experiment("Plant Leaf Disease")

with mlflow.start_run():
    mlflow.log_param('framework', 'pytorch')
    mlflow.log_param('data_split', '90/10')
    mlflow.log_param('type', 'DenseNet121')
    mlflow.log_params(config)
    train(model, train_loader, validation_loader, config, n_epochs=config['max_epochs'], stopping_treshold=15)

CUDA is available!  Training on GPU ...


C:\Users\ngrec\AppData\Local\pypoetry\Cache\virtualenvs\plant-leaf-diseases-identification-CBiGMIHu-py3.8\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch: 1/200, Training Loss: 0.921, Train accuracy 0.730 Validation Loss: 0.346, Validation accuracy 0.885
Epoch: 2/200, Training Loss: 0.644, Train accuracy 0.800 Validation Loss: 0.330, Validation accuracy 0.888
Epoch: 3/200, Training Loss: 0.604, Train accuracy 0.814 Validation Loss: 0.291, Validation accuracy 0.902
Epoch: 4/200, Training Loss: 0.590, Train accuracy 0.817 Validation Loss: 0.287, Validation accuracy 0.906
Epoch: 5/200, Training Loss: 0.570, Train accuracy 0.826 Validation Loss: 0.282, Validation accuracy 0.908
Epoch: 6/200, Training Loss: 0.554, Train accuracy 0.829 Validation Loss: 0.262, Validation accuracy 0.914
Epoch: 7/200, Training Loss: 0.560, Train accuracy 0.829 Validation Loss: 0.266, Validation accuracy 0.912
Epoch: 8/200, Training Loss: 0.536, Train accuracy 0.834 Validation Loss: 0.248, Validation accuracy 0.922
Epoch: 9/200, Training Loss: 0.551, Train accuracy 0.833 Validation Loss: 0.258, Validation accuracy 0.917
Epoch: 10/200, Training Loss: 0.535, 